<div style="text-align: right"> Madison Rusch </div>

<left>FINM 36700</left> 
<br>
<left>Autumn 2022</left>

<h2><center>Midterm #2 </center></h2>

### 1 Short Answer
#### 1.1 What is done in the construction of the value factor, HML, to reduce its correlation to MKT? SMB?
The HML, or value factor, is constructed by going long value stocks, and shorting growth stocks. By only using a certain decile at the top and bottom of the scale, we are able to reduce correlation to both MKT (market factor), and SMB (size factor). Additionally, the values used to sort the scale of the value and size factors are relatively orthogonal to each other, which also reduces correlation.

#### 1.2 The Value factor underperformed in the time since the DFA-case. What might DFA say in defense of value as a good investment, notwithstanding these statistics?
The value factor (HML) is lowly correlated to the Market and Size factors, which makes it a good diversifier for DFA, in spite of its low returns.


#### 1.3 We found these tangency portfolio weights for the 4 factor model considered by AQR. What might we conclude from this about this “AQR” pricing model?
We can see based on weighting of the factors that Market (MKT), Value (HML), and Momentum (UMD) factors are all roughly equal in importance in AQR's pricing model. However, the size factor (SMB) has such a low weighting in the tangency portfolio that it has very little impact in the model. 

#### 1.4 If the CAPM model worked perfectly, then the MKT factor would have the highest Sharpe ratio and highest Treynor ratio of all assets.
True. If the CAPM worked perfectly, then the market portfolio would be the tangency portfolio, and would therefore have the highest Sharpe ratio (by definition of the tangency portfolio). Additionally, the Treynor Ratio would equal the market returns (because the beta would = 1).

#### 1.5 What is the biggest difference in performance between long-only momentum and long-short momentum?
Though in general we saw higher means in the long-only momentum, it was due to that fact that the long-only portfolio was highly correlated to the market. The long-short momentum portfolio was negatively correlated to the market factor, and was much more diversifying when combined with the other factors.

#### 1.6 Do you think we could operate an effective momentum strategy given just BTC? Explain and be specific.
No. In order for a momentum strategy to be effective, it is required for there to be a large universe of assets to pick from, where we could take extreme winners and losers and construct a long-short portfolio. 

#### 1.7 In the factor model using MKT, SMB, HML, UMD, (the “AQR model”,) which factors can the entire market have a positive net exposure, and which must the market must have net 0 exposure?
The market is able to have positive net exposure for MKT, SMB, and UMD factors. However, not all investors can be exposed to the value (HML) factor, otherwise it would be reflected in the market premium. HML must therefore have net 0 exposure.

#### 1.8 The APT says that a ”perfect” Linear Factor Decomposition (LFD) will work as a perfect Linear Factor Pricing Model (LFPM). What must the factors achieve as an LFD to meet the APT’s condition?
In order to meet APT's condition, the LFD must have a "perfect" R-squared = 1.

#### 1.9 Is it possible for a factor model to fit perfectly as an LFPM yet to fit imperfectly as an LFD?
Yes. With Linear Factor Pricing Models, we are not trying to replicate portfolios. It's possible for our LFPM to have alphas of zeros, but an R-squared value less than one. This LFPM would therefore not be a perfect LFD, but would fit perfectly as an LFPM.


### 2 Pricing Model

In [77]:
# Some Python Code for importing packages/data
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
from statsmodels.regression.rolling import RollingOLS
import scipy
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.precision", 4)

# Read excel file, sheet
factor_data = pd.read_excel('../data/midterm_2.xlsx', 'factors (excess returns)')
factor_data = factor_data.set_index('Date')

returns_data = pd.read_excel('../data/midterm_2.xlsx', 'assets (excess returns)')
returns_data = returns_data.set_index('Date')

def time_series_test(df, factor_df, factors, annualization=12):
    columns = ['alpha']
    for factor in factors:
        columns.append(f'{factor} Beta')
    columns.append('R-squared')
        
    ts_test = pd.DataFrame(data = None, index = df.columns, columns = columns)

        
    for asset in ts_test.index:
        y = df[asset]
        X = sm.add_constant(factor_df[factors])

        reg = sm.OLS(y, X).fit()
        regParams = reg.params
        ts_test.loc[asset] = [regParams[0] * annualization, regParams[1], regParams[2], reg.rsquared]
    
    return ts_test

ts_test = time_series_test(returns_data, factor_data, ['MKT', 'RMW'])




#### 2.1 a) Report (for each asset) annualized alpha, beta, and r-squared.

In [78]:
display(ts_test)

,alpha,MKT Beta,RMW Beta,R-squared
BWX,-0.0406,0.2363,0.1024,0.2047
DBC,-0.052,0.5932,-0.1086,0.2568
EEM,-0.0689,0.9436,-0.0948,0.5601
EFA,-0.0582,0.9313,-0.0775,0.7622
HYG,-0.0004,0.4532,0.0362,0.5886
IEF,0.024,-0.0818,0.1693,0.082
IYR,0.001,0.9134,0.2731,0.5514
PSP,-0.0659,1.3408,-0.1098,0.8393
QAI,-0.0237,0.2782,0.0028,0.7174
SPY,0.0006,0.9629,0.1259,0.9924


#### 2.1 a) Report annualized MAE of the time-series alphas

In [79]:
print('MAE: ' + str(round(ts_test['alpha'].abs().mean(), 4)))

MAE: 0.0311


#### 2.1 a) Report mean of the r-squared statistics.

In [80]:
print('Mean of R-Squared: ' + str(round(ts_test['R-squared'].mean(), 4)))

Mean of R-Squared: 0.5155


#### 2.1 b) If the pricing model worked perfectly, what would these statistics be?
If the pricing model worked perfectly, we would expect the Mean Absolute Error to equal 0, but we would have no expectation for R-squared.

#### 2.1 c) What do you conclude from the time-series test of this model? That is, give some interpretation as to how well it prices the assets, and be specific.
This model seems to price the assets fairly well. We expect alphas to be zero if the pricing model worked perfectly, but given we're computing this on a sample, it's not suprising that we don't have alphas equal to zero. However, the alphas are generally close to zero, expecially for HYG and SPY. It seems this prices overall fairly well.

#### 2.2 Estimate the cross-sectional test of the pricing model. Include an intercept in the cross-sectional regression.
#### 2.2 a) Report the annualized intercept, annualized factor premia, r-squared, annualized mean-absolute error.

In [81]:
ts_test = ts_test.rename(columns={'MKT Beta': 'MKT', 'RMW Beta': 'RMW'})

y = returns_data.mean()
X = sm.add_constant(ts_test[['MKT','RMW']].astype(float))

cross_sect = sm.OLS(y, X).fit()
# display(cross_sect.summary())

print('Annualized Intercept: ' + str(round(cross_sect.params[0] * 12, 4)))
display((cross_sect.params[1:] * 12).to_frame('Annualized Factor Premia'))
print('R-squared: ' + str(round(cross_sect.rsquared, 4)))
print('Annualized MAE: ' + str(round(cross_sect.resid.abs().mean() * 12, 4)))

Annualized Intercept: -0.0214


,Annualized Factor Premia
MKT,0.1186
RMW,0.2101


R-squared: 0.9055
Annualized MAE: 0.0109


#### 2.2 b) If the pricing model worked perfectly, what would these statistics be?
If the pricing model worked perfectly, we would expect the R-squared of the cross-sectional test to equal 1 and the alpha to equal 0. We would have no expectations for the betas.

#### 2.2 c) What do you conclude from the cross-sectional test of this model? That is, give some interpretation as to how well it prices the assets, and be specific.
The cross-sectional seems to support our earlier argument that this pricing model is good not great. We have an alpha and MAE close to zero (-0.02 and 0.01 respectively). We also have an R-squared in the cross-sectional of 0.91. Given that we want the R-squared to equal 1, this model seems to do a pretty good job.

#### 2.3 Is the RMW factor more lucrative according to the cross-sectional or time-series estimates?
RMW seems to be more profitable according to the cross-sectional. It has a premia of 0.21 in the cross-sectional, and only one portfolio in the entire time-series results has a beta higher than that. In the time series results, RMW even has betas that go negative.

#### 2.4 Make a chart showing the annualized 11 asset premia, (that is, expected excess returns) as implied by the sample average, time-series, and cross-sectional.

In [82]:
premia_df = (returns_data.mean()*12).to_frame('Sample Average')

premia_df['Time Series Expected Returns'] = (factor_data.mean() * 12 * ts_test[['MKT','RMW']]).sum(axis = 1)

premia_df['Cross Sectional Expected Returns'] = (cross_sect.params[0] + (ts_test[['MKT','RMW']] * cross_sect.params[1:]).sum(axis=1)) * 12
display(premia_df)

,Sample Average,Time Series Expected Returns,Cross Sectional Expected Returns
BWX,-0.0051,0.0355,0.0281
DBC,0.0272,0.0792,0.0261
EEM,0.0592,0.1281,0.0706
EFA,0.0687,0.1269,0.0728
HYG,0.0633,0.0637,0.0399
IEF,0.0172,-0.0068,0.0044
IYR,0.1349,0.1339,0.1443
PSP,0.1168,0.1827,0.1146
QAI,0.0149,0.0386,0.0122
SPY,0.1373,0.1367,0.1193


### 3 Applications to Previously Seen Topics

#### 3.1 Calculate the tangency portfolio of the 11 excess return assets based on sample data. Report the tangency weights and sharpe ratio.

In [83]:
def compute_tangency(excessReturnMatrix):
    # Get the covariance matrix based on excess returns
    sigma = excessReturnMatrix.cov()
    
    # Get the number of asset classes (in this example should be 11)
    n = sigma.shape[0]
    
    # Get the vector of mean excess returns
    mu = excessReturnMatrix.mean()
    
    # Get sigma inverse
    sigma_inv = np.linalg.inv(sigma)
    
    # Now we have all the pieces, do the calculation
    weights = (sigma_inv @ mu) / (np.ones(n) @ sigma_inv @ mu)
    
    # Convert back to a Series for convenience
    return pd.Series(weights, index=mu.index)

tangency_portfolio = compute_tangency(returns_data)
tangency_portfolio.to_frame('Tangency Weights')

,Tangency Weights
BWX,-16.6834
DBC,2.6617
EEM,5.0023
EFA,-1.6528
HYG,15.8455
IEF,31.6714
IYR,-4.2510
PSP,-6.9802
QAI,-60.2653
SPY,33.1711


In [84]:
def portfolio_stats(excessReturnData, portfolio_weights):
    # Calculate the mean by multiplying the mean excess returns by the tangency weights and annualizing
    mean = excessReturnData.mean() @ portfolio_weights * 12

    # Volatility = sqrt(variance), and by class notes: variance = allocation_matrix * covariance_matrix * allocation_matrix
    # Annualize the result with sqrt(12)
    vol = np.sqrt(portfolio_weights @ excessReturnData.cov() @ portfolio_weights) * np.sqrt(12)

    # Sharpe Ratio is mean / vol
    sharpe_ratio = mean / vol

    # Format for easy reading
    return round(pd.DataFrame(data = [sharpe_ratio], 
        index = ['Sharpe'], 
        columns = ['Portfolio Stats']), 4)
    
display(portfolio_stats(returns_data, tangency_portfolio))

,Portfolio Stats
Sharpe,1.8413


#### 3.2 Re-do the previous question regarding the tangency portfolio. But this time, utilize the expected returns as implied by the cross-sectional estimation of the pricing model in the tangency portfolio estimation.

In [85]:
# cross_sectional_estimation = premia_df['Cross Sectional Expected Returns'].to_frame('Returns')
# cross_sectional_estimation.head()
# cross_sectional_tangency = compute_tangency(cross_sectional_estimation)
# display(cross_sectional_tangency)
cross_sectional_estimation = premia_df['Cross Sectional Expected Returns'] * returns_data * 12
cross_sectional_estimation.head()
cs_tangency = compute_tangency(cross_sectional_estimation)
cs_tangency.to_frame("Cross-Sectional Tangency Weights")


,Cross-Sectional Tangency Weights
BWX,-0.2453
DBC,0.0421
EEM,0.0293
EFA,-0.0094
HYG,0.1640
IEF,2.9472
IYR,-0.0122
PSP,-0.0252
QAI,-2.0442
SPY,0.1150


In [86]:
display(portfolio_stats(returns_data, cs_tangency))

,Portfolio Stats
Sharpe,0.2459


#### 3.3 Assume lognormal, iid returns. Suppose for this question that, (in log returns,) the expected excess return of RMW is 3% with a 7% volatility. What is the probability that RMW will have positive excess returns over the next 5 years?

In [87]:
# Mu calculation
mu = 0.03
var = 0.07

def prob_calc(h, tilde_mu, tilde_sigma):
    return scipy.stats.norm.cdf((- np.sqrt(h) * tilde_mu) / tilde_sigma)

def prob(h, tilde_mu, tilde_sigma):
    prob_df = pd.DataFrame(data = h, columns = ['h'])
    
    prob_df['Probability'] = prob_calc(prob_df['h'], tilde_mu, tilde_sigma)
    
    prob_df = prob_df.set_index('h')
    
    return prob_df

# print(prob_calc([5], mu, var))
probability = 1 - prob_calc([5], mu, var)[0]

print(f"The probability that RMW will have positive excess returns is {round(100* probability, 2)}%")

The probability that RMW will have positive excess returns is 83.1%
